**Домашнее задание к лекции "Основы веб-скрапинга"**

**Обязательная часть**

Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>
В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [ ]:
import pandas as pd
import requests

In [ ]:
# Сформируем поисовый запрос.

habr_res = requests.get('https://habr.com/ru/search/?q=json&target_type=posts&order=relevance')
habr_res

<Response [200]>

In [ ]:
# Импорт библиотеки BeautifulSoup и создание на ее основе объекта soup:
from bs4 import BeautifulSoup
#soup = BeautifulSoup(habr_res.text)
#soup

In [ ]:
# Библиотека time - для создания задержки между запросами.
import time

def habr_search(query):
    habr_df = pd.DataFrame()
    for el in query:
      # Чтобы получать русскоязычные ресурсы, URL заишем в следующем виде (согласно записи Request URL в DevTools):
      URL = 'https://habr.com/ru/search/'
      # Параметр для обозначения строки поискового запроса на сайте habr.com обозначен буквой q, остальные параметры оставим по умолчанию:
      params = {'q': el}
      req = requests.get(URL, params=params)
      time.sleep(0.3)
      soup = BeautifulSoup(req.text)
      articles = soup.find_all('article', class_='tm-articles-list__item')
      for article in articles:
        # Чтобы в датафрейме все даты были в едином формате, обратимся к атрибуту datetime и возьмем его часть [0:10] (не захватывая время):
        date = article.find('time').get('datetime')[0:10]
        title = article.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').text
        link_1 = article.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').find('a').get('href')
        # Чтобы ссылки в датафрейм добавлялись полноценные:
        link = 'https://habr.com' + link_1
        row = {'Дата': date, 'Заголовок': title, 'Ссылка_на_материал': link}
        habr_df = pd.concat([habr_df, pd.DataFrame([row])])
    # Чтобы материалы в датафрейме не дублировались, удалим дубликаты; а также сбросим индексы (нумерацию строк):
    return habr_df.drop_duplicates().reset_index(drop=True)

query = ['python', 'python для инженеров', 'data science']
habr_search(query)
# Результирующий датафрейм содержит меньше 60строк, значит дубликаты строк удалились.

,Дата,Заголовок,Ссылка_на_материал
0,2021-12-13,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/company/southbridge/news/t...
1,2022-01-20,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...
2,2020-04-21,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/
3,2021-07-06,Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/company/raiffeisenbank/new...
4,2022-01-13,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/company/southbridge/news/t...
5,2020-12-04,Python как компилируемый статически типизирова...,https://habr.com/ru/news/t/531402/
6,2022-03-08,Вышел мартовский релиз расширения Python для V...,https://habr.com/ru/news/t/654707/
7,2020-03-03,В начале этого года Python сместил Java и стал...,https://habr.com/ru/company/itsumma/news/t/490...
8,2021-11-16,EPAM разработала бесплатный курс по программир...,https://habr.com/ru/company/epam_systems/news/...
9,2022-08-19,Осталась неделя до старта 4 потока Python для ...,https://habr.com/ru/company/southbridge/news/t...


**Дополнительная часть (необязательная)**

Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [ ]:
import time
# Функция будет принимать два параметра: список поисковых запросов и количество страниц:
def habr_search_3(query, pages):
    habr_df = pd.DataFrame()
    # Для каждого поискового запроса из списка query будем проходить всё заданное количество страниц pages, формируя строки с заданными столбцами и добавляя их в датафрейм.
    for el in query:

      for page_num in range(1, pages + 1):
        # Т.к. номер страницы имеется в request url (https://habr.com/ru/search/page1...), то задавать номер страницы будем, изменяя число после слова page,
        # оформим это с помощью f-строки:
        URL = f"https://habr.com/ru/search/page{page_num}/"

        params = {'q': el}

        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='tm-articles-list__item')
        for article in articles:
          date = article.find('time').get('datetime')[0:10]
          title = article.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').text

          link = 'https://habr.com' + article.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').find('a').get('href')
          req_text = requests.get(link).text
          soup_text = BeautifulSoup(req_text)
          time.sleep(0.3)
          # Чтобы обойтии проблему, вследствие которой не открываются полные тексты некоторых постов, попавших в результаты поиска, как, например,
          # пост "EPAM разработала бесплатный курс по программированию" (строка №8 в датафрейме), воспользуемся инструкцией 'try-except':
          try: full_text = soup_text.find('div', class_ = 'article-formatted-body').text.strip()
          except: full_text = '-'

          likes = article.find('span', class_ = 'tm-votes-meter__value').text

          row = {'Дата': date, 'Заголовок': title, 'Ссылка_на_материал': link, 'Текст_материала' : full_text, 'Количество_лайков' : likes}
          habr_df = pd.concat([habr_df, pd.DataFrame([row])])

    return habr_df.drop_duplicates().reset_index(drop=True)

# Зададим в качестве параметров функции habr_search_3 список из 2-х поисковых запросов и количество страниц, равное трем:
data_2 = habr_search_3(['python', 'python для инженеров'], 3)
data_2
# Общее количество строк меньше 120 (20 постов х 2 поисковых запроса х 3 страницы), т.к. удалены дубликаты.

,Дата,Заголовок,Ссылка_на_материал,Текст_материала,Количество_лайков
0,2021-12-13,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/company/southbridge/news/t...,21 декабря Слёрм проведёт открытый урок «ChatO...,+9
1,2022-01-20,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...,"Курс нацелен дать максимальную пользу, поэтому...",+10
2,2020-04-21,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/,"20 апреля 2020 года, спустя почти десять лет п...",+19
3,2021-07-06,Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/company/raiffeisenbank/new...,"Первый открытый онлайн-митап сообщества, для к...",+3
4,2022-01-13,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/company/southbridge/news/t...,19 января Слёрм проведёт открытый урок «Пишем ...,+10
...,...,...,...,...,...
104,2022-04-05,CLD — Open source проект для ИТ компаний и SRE...,https://habr.com/ru/post/659023/,CLD это система для обеспечения комплексной ин...,+5
105,2015-01-05,"SageMathCloud — мечта для любителей Python, ма...",https://habr.com/ru/post/247277/,SageMathCloud (сокращённо SMC) — это онлайновы...,+49
106,2018-12-14,Методика оценки знаний инженера. Путь архитект...,https://habr.com/ru/post/433226/,Можно ли на одной картинке представить уровень...,+16
107,2022-03-14,"Рисуем красивые трейсбеки, перехватывая исключ...",https://habr.com/ru/company/wunderfund/blog/65...,"Все мы тратим немало времени на отладку, копая...",+27
